In [1]:
import requests
from bs4 import BeautifulSoup
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time

In [2]:
#Pulp Fiction script
response = requests.get('https://www.dailyscript.com/scripts/pulp_fiction.html')
soup = BeautifulSoup(response.text, 'html.parser')
text = soup.find('pre').getText()
text = text[524:]

#unique characters
vocab = sorted(set(text))

In [3]:
#represent text as numeric ids
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

#translate numeric ids back to chracaters
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

#joins individual characters back into readable text
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [4]:
#convert entire script into numeric ids
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids))
    #print(chars_from_ids(ids).numpy().decode('utf-8'))


tf.Tensor(b'I', shape=(), dtype=string)
tf.Tensor(b'N', shape=(), dtype=string)
tf.Tensor(b'T', shape=(), dtype=string)
tf.Tensor(b'.', shape=(), dtype=string)
tf.Tensor(b' ', shape=(), dtype=string)
tf.Tensor(b'C', shape=(), dtype=string)
tf.Tensor(b'O', shape=(), dtype=string)
tf.Tensor(b'F', shape=(), dtype=string)
tf.Tensor(b'F', shape=(), dtype=string)
tf.Tensor(b'E', shape=(), dtype=string)


In [5]:
#break the script into chunks of seq_length+1
#for example, say seq_length is 4 and our text is "Hello"
#The input sequence would be "Hell", and the target sequence "ello"

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(3):
  #print(chars_from_ids(seq))
  print(text_from_ids(seq).numpy())

b"INT. COFFEE SHOP \xc2\x96 MORNING\r\n\r\n               A normal Denny's, Spires-like coffee shop in Los Angeles"
b". \r\n               It's about 9:00 in the morning. While the place isn't jammed, \r\n               the"
b"re's a healthy number of people drinking coffee, munching \r\n               on bacon and eating eggs.\r"


In [6]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target('do you speak it?')

('do you speak it', 'o you speak it?')

In [7]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"INT. COFFEE SHOP \xc2\x96 MORNING\r\n\r\n               A normal Denny's, Spires-like coffee shop in Los Angele"
Target: b"NT. COFFEE SHOP \xc2\x96 MORNING\r\n\r\n               A normal Denny's, Spires-like coffee shop in Los Angeles"


In [8]:
BATCH_SIZE = 64

# buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

This model has three layers:

**tf.keras.layers.Embedding:** The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;

**tf.keras.layers.GRU:** A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)

**tf.keras.layers.Dense:** The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [9]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [10]:
#length of the vocabulary in chars
vocab_size = len(vocab)

#embedding dimension
embedding_dim = 256

#number of RNN units
rnn_units = 1024

In [11]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [12]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 82) # (batch_size, sequence_length, vocab_size)


In [13]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  20992     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  84050     
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [14]:
#sample prediction on untrained model
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'                         A "please" would be nice.\r\n\r\n               The Wolf stops and turns around'

Next Char Predictions:
 b'O:OxxKE52kw1sL/Sg6so0vRGQ?k#S\rp#FB mXUjBJV?Sq?i8W#iDkV\xc2\x96JO5grXVBPkfrNxl&!LQ8KNSVGZ"uHq:LFu kh6p/\n5EiD'


In [15]:
#save models during training
checkpoint_dir = './pulp_fiction_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=False) #save entire model, not just the weights

In [17]:
#assign loss and optimizer
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)
EPOCHS = 100

In [18]:
#train
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
47/47 [==============================] - 217s 5s/step - loss: 2.7607


INFO:tensorflow:Assets written to: ./pulp_fiction_training_checkpoints/ckpt_1/assets


INFO:tensorflow:Assets written to: ./pulp_fiction_training_checkpoints/ckpt_1/assets


Epoch 2/100
47/47 [==============================] - 213s 5s/step - loss: 1.8080


INFO:tensorflow:Assets written to: ./pulp_fiction_training_checkpoints/ckpt_2/assets


...

INFO:tensorflow:Assets written to: ./pulp_fiction_training_checkpoints/ckpt_99/assets


INFO:tensorflow:Assets written to: ./pulp_fiction_training_checkpoints/ckpt_99/assets


Epoch 100/100
47/47 [==============================] - 217s 5s/step - loss: 0.0743


INFO:tensorflow:Assets written to: ./pulp_fiction_training_checkpoints/ckpt_100/assets


INFO:tensorflow:Assets written to: ./pulp_fiction_training_checkpoints/ckpt_100/assets


In [19]:
#make a single step prediction
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [20]:
model = OneStep(model, chars_from_ids, ids_from_chars)

In [23]:
start = time.time()
states = None
next_char = tf.constant(["I'm pretty far from ok."]) #starting text
result = [next_char]

for n in range(1000):
    next_char, states = model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

I'm pretty far from ok. 
                         So let's got a copporatop to be a 
                         fuckin' special shit.

                                     MIA
                         Who's Antwan?

                                     JIMMIE
                         Ereke, your ass if you want to 
                         Pack of Red.

                                     BUTCH
                         Where's the shoe stores?

                                     BUTCH
                              (to Fabienne)
                         Som: It a glad to get burters 
                         tell this fuckin' asshole right now!
                              (he approached her)
                         Royale with Cheese. What'd the car 
                         workin' off the rearview with the mone of the 
                         patio of his ass-littly giol. But then he 
               Vincent and Jules standing in this siendef.

                                    